<a href="https://colab.research.google.com/github/gelso2210/CS_course/blob/main/lecture_py_07_groupby_apply_idxmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Group by


In [1]:
import pandas as pd

We will work on the `orders` dataframe.

In [2]:
orders = pd.read_csv("https://github.com/gdv/foundationsCS/raw/main/students/ex-data/Northwind/Orders.csv")

In [4]:
orders.head()

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10248,VINET,5,2012-07-04,2012-08-01,2012-07-16,3,16.75,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,Western Europe,51100,France
1,10249,TOMSP,6,2012-07-05,2012-08-16,2012-07-10,1,22.25,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
2,10250,HANAR,4,2012-07-08,2012-08-05,2012-07-12,2,25.00,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,South America,05454-876,Brazil
3,10251,VICTE,3,2012-07-08,2012-08-05,2012-07-15,1,20.25,Victuailles en stock,"2, rue du Commerce",Lyon,Western Europe,69004,France
4,10252,SUPRD,4,2012-07-09,2012-08-06,2012-07-11,2,36.25,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium


## For each customer, compute the number of orders

In [5]:
orders.groupby('CustomerId').size()

,0
CustomerId,
ALFKI,200
ANATR,192
ANTO,7
ANTON,191
AROUT,162
...,...
WARTH,192
WELLI,185
WHITC,170


A more flexible, but a bit cumbersome, alternative is `count`

In [6]:
orders.groupby('CustomerId').count()

,Id,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
CustomerId,,,,,,,,,,,,,
ALFKI,200,200,200,200,200,200,200,200,200,200,200,198,200
ANATR,192,192,192,192,192,192,192,192,192,192,192,188,192
ANTO,7,7,7,7,7,7,7,7,7,7,7,7,7
ANTON,191,191,191,191,191,191,191,191,191,191,191,190,191
AROUT,162,162,162,162,162,162,162,162,162,162,162,161,162
...,...,...,...,...,...,...,...,...,...,...,...,...,...
WARTH,192,192,192,192,192,192,192,192,192,192,192,190,192
WELLI,185,185,185,185,185,185,185,185,185,185,185,183,185
WHITC,170,170,170,170,170,170,170,170,170,170,170,168,170


Since `count` counts the number of non-missing rows for each column, it is better to choose a column.

In [7]:
orders.groupby('CustomerId').count()['Id']

,Id
CustomerId,
ALFKI,200
ANATR,192
ANTO,7
ANTON,191
AROUT,162
...,...
WARTH,192
WELLI,185
WHITC,170


The result of a `groupby` moves the column in the `groupby` to the index. Notice the set of columns that are extracted.

In [38]:
orders.groupby('CustomerId', as_index = False).count()[['CustomerId','Id']]

<ipython-input-38-af7222616696>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  orders.groupby('CustomerId', as_index = False).count()[['CustomerId','Id']][orders['CustomerId'] == 'ALFKI']


,CustomerId,Id


Can we use a `groupby` by itself?

In [13]:
orders.groupby('CustomerId')

In [17]:
orders.value_counts('CustomerId')  # probabily the best method? dose not to group by anything

,count
CustomerId,
SAVEA,218
LAMAI,217
QUICK,210
VAFFE,207
BERGS,206
...,...
TRAIH,142
QUEE,13
ANTO,7


# idxmax - idxmin

We need the `Products` table

In [79]:
products = pd.read_csv("https://github.com/gdv/foundationsCS/raw/main/students/ex-data/Northwind/Products.csv")

In [40]:
products.head()

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
0,1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
1,2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
2,3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
3,4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
4,5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


Extract the maximum of the `UnitsInStock` column

In [47]:
products.loc[products.UnitsInStock==products['UnitsInStock'].max()]

,Id,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
74,75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,0


Find the product with maximum number of units in stock

In [43]:
products['UnitsInStock'].idxmax()

74

In [66]:
products.iloc[products['UnitsInStock'].idxmax()]

,74
Id,75
ProductName,Rhönbräu Klosterbier
SupplierId,12
CategoryId,1
QuantityPerUnit,24 - 0.5 l bottles
UnitPrice,7.75
UnitsInStock,125
UnitsOnOrder,0
ReorderLevel,25
Discontinued,0


Let us investigate what happens if we also have an explicit index

In [81]:
products.set_index('Id', inplace = True)
products.head()

,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
Id,,,,,,,,,
1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0
2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0
3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0
4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0
5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1


In [68]:
products['UnitsInStock'].idxmax()

75

In [69]:
products.iloc[products['UnitsInStock'].idxmax()]

,76
ProductName,Lakkalikööri
SupplierId,23
CategoryId,1
QuantityPerUnit,500 ml
UnitPrice,18.0
UnitsInStock,57
UnitsOnOrder,0
ReorderLevel,20
Discontinued,0


In [71]:
products.loc[products['UnitsInStock'].idxmax()]  # questo è più sensato in quanto ho settato  un ID "custom"

,75
ProductName,Rhönbräu Klosterbier
SupplierId,12
CategoryId,1
QuantityPerUnit,24 - 0.5 l bottles
UnitPrice,7.75
UnitsInStock,125
UnitsOnOrder,0
ReorderLevel,25
Discontinued,0


`idxmax` and `idxmin` return the **explicit** index, if possible.

#### What if there are several products achieving the minimum?

In [78]:
products.loc[products['UnitsInStock'].idxmin()]

,17
ProductName,Alice Mutton
SupplierId,7
CategoryId,6
QuantityPerUnit,20 - 1 kg tins
UnitPrice,39.0
UnitsInStock,0
UnitsOnOrder,0
ReorderLevel,0
Discontinued,1


In [82]:
products[products['UnitsInStock'] == products['UnitsInStock'].min()]
# it takes the first in the order of the index

,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
Id,,,,,,,,,
5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1
17,Alice Mutton,7,6,20 - 1 kg tins,39.00,0,0,0,1
29,Thüringer Rostbratwurst,12,6,50 bags x 30 sausgs.,123.79,0,0,0,1
31,Gorgonzola Telino,14,4,12 - 100 g pkgs,12.50,0,70,20,0
53,Perth Pasties,24,6,48 pieces,32.80,0,0,0,1


### Maximum by group

For each category, find a product with minimum number of units in stock.

In [83]:
products.groupby('CategoryId').idxmin()

,ProductName,SupplierId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued
CategoryId,,,,,,,,
1,1,1,1,24,70,1,24,1
2,3,3,8,3,5,4,4,3
3,48,16,47,19,21,16,20,16
4,60,11,11,33,31,12,12,11
5,52,22,23,52,56,22,42,22
6,17,9,54,54,17,9,9,54
7,74,7,7,74,74,7,14,7
8,40,10,30,13,45,10,10,10


Extract the relevant columns

In [84]:
products.groupby('CategoryId')['UnitsInStock'].idxmin()

,UnitsInStock
CategoryId,
1,70
2,5
3,21
4,31
5,56
6,17
7,74
8,45


It is more complex to extract all products with minimum number of units in stock.

# Apply

Column- or table-wise operators are much more efficient.

What if we want to create a new column with the result of a complex expression?

Add a column `alert` that is equal to the sum of `UnitsOnOrder` and `UnitsInStock`, but is equal to 100 if that sum is larger than 100 and is equal to 0 if the product is discontinued.

First we build a function that computes the value of `alert` given a row of the dataframe.

In [86]:
def alert(row):
    if row['Discontinued'] > 0:
        return 0
    return min(row['UnitsOnOrder'] + row['UnitsInStock'], 100)

Then we can `apply` the function to all rows of the dataframe.

In [87]:
products['alert'] = products.apply(alert, axis = 1)
products

,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,alert
Id,,,,,,,,,,
1,Chai,1,1,10 boxes x 20 bags,18.00,39,0,10,0,39
2,Chang,1,1,24 - 12 oz bottles,19.00,17,40,25,0,57
3,Aniseed Syrup,1,2,12 - 550 ml bottles,10.00,13,70,25,0,83
4,Chef Anton's Cajun Seasoning,2,2,48 - 6 oz jars,22.00,53,0,0,0,53
5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
73,Röd Kaviar,17,8,24 - 150 g jars,15.00,101,0,5,0,100
74,Longlife Tofu,4,7,5 kg pkg.,10.00,4,20,5,0,24
75,Rhönbräu Klosterbier,12,1,24 - 0.5 l bottles,7.75,125,0,25,0,100


In [88]:
products[products['Discontinued'] > 0]

,ProductName,SupplierId,CategoryId,QuantityPerUnit,UnitPrice,UnitsInStock,UnitsOnOrder,ReorderLevel,Discontinued,alert
Id,,,,,,,,,,
5,Chef Anton's Gumbo Mix,2,2,36 boxes,21.35,0,0,0,1,0
9,Mishi Kobe Niku,4,6,18 - 500 g pkgs.,97.00,29,0,0,1,0
17,Alice Mutton,7,6,20 - 1 kg tins,39.00,0,0,0,1,0
24,Guaraná Fantástica,10,1,12 - 355 ml cans,4.50,20,0,0,1,0
28,Rössle Sauerkraut,12,7,25 - 825 g cans,45.60,26,0,0,1,0
29,Thüringer Rostbratwurst,12,6,50 bags x 30 sausgs.,123.79,0,0,0,1,0
42,Singaporean Hokkien Fried Mee,20,5,32 - 1 kg pkgs.,14.00,26,0,0,1,0
53,Perth Pasties,24,6,48 pieces,32.80,0,0,0,1,0
